# import

In [95]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import warnings

from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

# config

In [103]:
warnings.filterwarnings("ignore")

INPUT = "../data/input"
OUTPUT = "../data/output"
NAME = "baseline001"

# load data

In [105]:
data_train = pd.read_csv(os.path.join(INPUT, "train.csv"))
data_test = pd.read_csv(os.path.join(INPUT, "test.csv"))
data = pd.concat([data_train, data_test])

In [77]:
data.head()

,PassengerId,Perished,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,1.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,0.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,0.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,0.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,1.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [78]:
data.isnull().sum()

PassengerId       0
Perished        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

# preprocess

In [79]:
# 欠損値の除去
data = pd.concat([train, test], sort=False)
data["Age"].fillna(data["Age"].mean(), inplace=True)
data["Fare"].fillna(data["Fare"].mean(), inplace=True)
data.drop('Cabin', axis=1, inplace=True)
data["Embarked"].fillna("S", inplace=True)

In [80]:
data.isnull().sum()

PassengerId      0
Perished       418
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         0
dtype: int64

# feature engineering

In [81]:
sex_onehot = pd.get_dummies(data["Sex"])
sex_onehot.head()

,female,male
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1


In [82]:
embarked_onehot = pd.get_dummies(data["Embarked"])
embarked_onehot.head()

,C,Q,S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1


In [83]:
pclass = pd.DataFrame(data["Pclass"])
pclass.head()

,Pclass
0,3
1,1
2,3
3,1
4,3


In [84]:
age = pd.DataFrame(data['Age'])
age.head()

,Age
0,22.0
1,38.0
2,26.0
3,35.0
4,35.0


In [85]:
sibsp = pd.DataFrame(data["SibSp"])
sibsp.head()

,SibSp
0,1
1,1
2,0
3,1
4,0


In [86]:
parch = pd.DataFrame(data["Parch"])
parch.head()

,Parch
0,0
1,0
2,0
3,0
4,0


In [87]:
fare = pd.DataFrame(data["Fare"])
fare.head()

,Fare
0,7.2500
1,71.2833
2,7.9250
3,53.1000
4,8.0500


In [88]:
pclass = pd.DataFrame(data["Pclass"])
pclass.head()

,Pclass
0,3
1,1
2,3
3,1
4,3


In [89]:
features_list = [sex_onehot,
            embarked_onehot,
            sibsp,
            age,
            pclass,
            parch,
            fare]

In [90]:
features = pd.concat(features_list, axis=1)

In [91]:
features.head()

,female,male,C,Q,S,SibSp,Age,Pclass,Parch,Fare
0,0,1,0,0,1,1,22.0,3,0,7.2500
1,1,0,1,0,0,1,38.0,1,0,71.2833
2,1,0,0,0,1,0,26.0,3,0,7.9250
3,1,0,0,0,1,1,35.0,1,0,53.1000
4,0,1,0,0,1,0,35.0,3,0,8.0500


In [94]:
X_train = features[:len(data_train)]
y_train = data_train["Perished"]
X_test = features[len(data_train):]

# model

In [129]:
clf = RandomForestClassifier(max_depth=7, min_samples_leaf=2, random_state=0)
clf.fit(X_train, y_train)
score = clf.score(X_train, y_train)
print(clf.score(X_train, y_train))
pred = clf.predict(X_test)

0.8832772166105499


# submission

In [140]:
submission = pd.read_csv(os.path.join(OUTPUT, "sub_sample.csv"))
submission["perished"] = pred
now = datetime.datetime.now()
submission.to_csv(os.path.join(OUTPUT, "sub_{0:%Y%m%d%H%M}_{1}.csv").format(now, round(score, 2)))